# Tasks
1. Data retrival 
- Obtain MovieLens 1M dataset [x]
- Load dateset [x]
- Check integrity of data [] - 69 movies missing, User ID, zip-codes have sometimes addition, ratings will rate non-existent movies
- User-Item Interaction Matrix [ ]
- Split data for 5 fold cross-validation [X]
- Handling the cold starts [ ]
2. Recomendation algorithms  [ ]
- Naive approach [X]
- UV matrix decomposition [ ]
- Matrix Factorization [ ]
- For each calculate: [ ] 
- - RMSE (Root mean SQuared Error) and MAE (Mean Absolute Error) [ ]
- - Handling Cold starts [ ]
3. Visulalzation [ ] 
- PCA [ ]
- t-SNE [ ]
- UMAP[ ]

In [78]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.linear_model import LinearRegression

## Data retrival

In [79]:
# Loading data sets

# movies
df_movies = pd.read_csv("ml-1m/movies.dat", sep='::', encoding='ISO-8859-1', header=None, engine='python', names=['MovieID', 'Title', 'Genres'])
df_movies = df_movies.rename({0: 'MovieID', 1: 'Title', 2: 'Genre'}, axis='columns')

#ratings
df_ratings = pd.read_csv("ml-1m/ratings.dat", sep='::', encoding='ISO-8859-1', header=None, engine='python', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

#users
df_users = pd.read_csv("ml-1m/users.dat", sep='::', encoding='ISO-8859-1', header=None, engine="python", names=['UserID', 'Gender', 'Age', 'Occupation', 'ZipCode'])
df_users.columns = ['UserID', 'Gender', 'Age', 'Occupation', 'ZipCode']



In [80]:
# Checking data integrity

# movies
print(df_movies.head())
print

# ratings
print(df_ratings.head())
print

# users

print(df_users.head())

# In movies.dat there is missing movieID of 91 we create a placeholder
new_movie = pd.DataFrame({'MovieID': [91], 'Title': ['Unknown'], 'Genres': ['Unknown']})
df_movies = pd.concat([df_movies, new_movie], ignore_index=True)




   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
   UserID Gender  Age  Occupation ZipCode
0       1      F    1          10   48067
1       2      M   56          16   70072
2       3      M   25          15   55117
3       4      M   45           7   02460
4       5      M   25          20   55455


In [81]:
# User-item interaction

In [82]:
# Users split into 5

num_folds = 5

# Kfold object to split data into
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Convert the DataFrame to a list to ensure consistent indices
user_data = df_users.values
ratings_data = df_ratings.values

# Split the data into 5 folds
for fold, (train_indices, test_indices) in enumerate(kf.split(user_data)):
    # Split the users dataset
    train_users = df_users.iloc[train_indices]
    test_users = df_users.iloc[test_indices]

    # Split the ratings dataset
    train_ratings = df_ratings[df_ratings['UserID'].isin(train_users['UserID'])]
    test_ratings = df_ratings[df_ratings['UserID'].isin(test_users['UserID'])]

    print(f"Fold {fold + 1} - Train Users: {len(train_users)}, Test Users: {len(test_users)}")


Fold 1 - Train Users: 4832, Test Users: 1208
Fold 2 - Train Users: 4832, Test Users: 1208
Fold 3 - Train Users: 4832, Test Users: 1208
Fold 4 - Train Users: 4832, Test Users: 1208
Fold 5 - Train Users: 4832, Test Users: 1208


In [83]:
# Movie split into 5
num_folds = 5

# Kfold object to split data into
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Convert the DataFrame to a list to ensure consistent indices
movie_data = df_movies.values

# Split the data into 5 folds
for fold, (train_indices, test_indices) in enumerate(kf.split(movie_data)):
    # Split the movies dataset
    train_movies = df_movies.iloc[train_indices]
    test_movies = df_movies.iloc[test_indices]

    print(f"Fold {fold + 1} - Train Movies: {len(train_movies)}, Test Movies: {len(test_movies)}")

Fold 1 - Train Movies: 3107, Test Movies: 777
Fold 2 - Train Movies: 3107, Test Movies: 777
Fold 3 - Train Movies: 3107, Test Movies: 777
Fold 4 - Train Movies: 3107, Test Movies: 777
Fold 5 - Train Movies: 3108, Test Movies: 776


In [84]:
# Ratins into 5
num_folds = 5

# Kfold object to split data into
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Convert the DataFrame to a list to ensure consistent indices
ratings_data = df_ratings.values

# Split the data into 5 folds
for fold, (train_indices, test_indices) in enumerate(kf.split(ratings_data)):
    # Split the ratings dataset
    train_ratings = df_ratings.iloc[train_indices]
    test_ratings = df_ratings.iloc[test_indices]

    print(f"Fold {fold + 1} - Train Ratings: {len(train_ratings)}, Test Ratings: {len(test_ratings)}")


Fold 1 - Train Ratings: 800167, Test Ratings: 200042
Fold 2 - Train Ratings: 800167, Test Ratings: 200042


Fold 3 - Train Ratings: 800167, Test Ratings: 200042
Fold 4 - Train Ratings: 800167, Test Ratings: 200042
Fold 5 - Train Ratings: 800168, Test Ratings: 200041


# Recommendations algorithms

## Naive approach

In [85]:
class recommenderSystem():
    def Naive_1(self, df_ratings):
        # Naive Approach
        r_global = df_ratings['Rating'].mean()
        r_item = df_ratings.groupby('MovieID')['Rating'].mean().reset_index().rename({'Rating': 'R_item'}, axis='columns')
        r_user = df_ratings.groupby('UserID')['Rating'].mean().reset_index().rename({'Rating': 'R_user'}, axis='columns')

        df_ratings = df_ratings.merge(r_item, on=['MovieID']).merge(r_user, on=['UserID'])
        print(r_global)
        print(r_item.head())
        print(r_user.head())
        print(df_ratings.head())

        X = df_ratings[['R_item', 'R_user']]
        y = df_ratings['Rating']
        model = LinearRegression().fit(X, y)

        alpha, beta = model.coef_
        gamma = model.intercept_

        print(f'alpha: {alpha}, beta: {beta}, gamma: {gamma}')


In [86]:
rec = recommenderSystem()
rec.Naive_1(df_ratings)

3.581564453029317
   MovieID    R_item
0        1  4.146846
1        2  3.201141
2        3  3.016736
3        4  2.729412
4        5  3.006757
   UserID    R_user
0       1  4.188679
1       2  3.713178
2       3  3.901961
3       4  4.190476
4       5  3.146465
   UserID  MovieID  Rating  Timestamp    R_item    R_user
0       1     1193       5  978300760  4.390725  4.188679
1       1      661       3  978302109  3.464762  4.188679
2       1      914       3  978301968  4.154088  4.188679
3       1     3408       4  978300275  3.863878  4.188679
4       1     2355       5  978824291  3.854375  4.188679
alpha: 0.8757397042780238, beta: 0.7821285278627704, gamma: -2.356197475012707
